In [1]:
from pathlib import Path
import pandas as pd
from collections import defaultdict

DATA = Path('../data/version-3/')

In [2]:
def clean_colnames(df):
    df.columns = [col.replace(' ', '-').lower() for col in df.columns]

# A. Lexicons

### Food Lexicon

In [3]:
df = pd.read_csv(DATA/'food-lexicon-automated.tsv', sep='\t', encoding='utf-8')
clean_colnames(df)

del df['foodb-group'], df['foodb-subgroup']
df.head()

,common-names,food-id,food-category,food-name,scientific-name,tax-id
0,alexanders; horse parsley,Plant ID:40962,Miscellaneous,Alexanders,Smyrnium olusatrum,40962.0
1,NaN,Plant ID:942083,Miscellaneous,Scrophularia umbrosa,Scrophularia umbrosa,942083.0
2,dotted blazing star,Plant ID:344074,Miscellaneous,Dotted blazing star,Liatris punctata,344074.0
3,plymouth pear,Plant ID:761867,Miscellaneous,Plymouth pear,Pyrus cordata,761867.0
4,NaN,Plant ID:49166,Miscellaneous,Rhododendron kaempferi,Rhododendron kaempferi,49166.0


In [4]:
# Replace NaN with None
df = df.where((pd.notnull(df)), None)

# Rename columns to match db schema.
df.rename(columns={
    'common-names': 'common_names',
    'food-name': 'display_name',
    'food-id': 'food_id',
    'food-category': 'food_category',
    'scientific-name': 'scientific_name',
    'tax-id':'tax_id',
}, inplace=True)

data_list= list(df.T.to_dict().values())
data_list[0]

{'common_names': 'alexanders; horse parsley',
 'food_id': 'Plant ID:40962',
 'food_category': 'Miscellaneous',
 'display_name': 'Alexanders',
 'scientific_name': 'Smyrnium olusatrum',
 'tax_id': 40962.0}

### Disease Lexicon

In [5]:
df = pd.read_csv(DATA/'disease-lexicon.tsv', sep='\t', encoding='utf-8', index_col=0)
clean_colnames(df)
df.reset_index(inplace=True, drop=True)
df.head()

,disease-id,disease-name,other-ids,parent-ids,disease-treenum,disease-synonyms,disease-category
0,MESH:D008105,"Liver Cirrhosis, Biliary",DO:DOID:12236|OMIM:109720|OMIM:614220|OMIM:614221,MESH:D002780|MESH:D008103,C06.130.120.135.250.250|C06.552.150.250|C06.55...,"Biliary Cirrhosis|Biliary Cirrhosis, Primary|B...",Digestive system disease
1,MESH:D009369,Neoplasms,DO:DOID:162,MESH:C,C04,Benign Neoplasm|Benign Neoplasms|Cancer|Cancer...,Cancer
2,MESH:D005892,"Gingivitis, Necrotizing Ulcerative",DO:DOID:13924|DO:DOID:9673,MESH:D005674|MESH:D005891|MESH:D007239,C01.252.400.388.350.400|C01.539.424|C07.465.71...,Acute Membranous Gingivitides|Acute Membranous...,Bacterial infection or mycosis|Mouth disease
3,MESH:D010020,Osteonecrosis,DO:DOID:10159,MESH:D001847|MESH:D009336,C05.116.852|C23.550.717.732,Aseptic Necrosis of Bone|Avascular Necrosis of...,Musculoskeletal disease|Pathology (process)
4,MESH:D014522,Urethral Diseases,DO:DOID:732,MESH:D014570,C12.777.767|C13.351.968.767,"Diseases, Urethral|Disease, Urethral|Urethral ...",Urogenital disease (female)|Urogenital disease...


In [6]:
df = df.where((pd.notnull(df)), None)

# Rename to match schema
df.rename(columns={
    'disease-id': 'disease_id',
    'disease-name': 'disease_name',
    'disease-category': 'disease_category',
    'disease-synonyms': 'disease_synonyms'
}, inplace=True)

# Keep only selected columns.
df = df[['disease_id', 'disease_name', 'disease_category']]

data_list= list(df.T.to_dict().values())

### Gene Lexicon

In [7]:
df = pd.read_csv('../data/version-3/gene-disease.tsv', sep='\t', encoding='utf-8')[['gene-id']]
df.drop_duplicates(subset=['gene-id'], inplace=True)
df.head()

,gene-id
0,1588
1,26504
2,9499
3,29940
4,113189


In [8]:
df.rename(columns={
    'gene-id': 'gene_id',
}, inplace=True)


data_list= list(df.T.to_dict().values())

### References

In [9]:
df = pd.read_csv(DATA/'publication-records.tsv', sep='\t', encoding='utf-8', index_col='Unnamed: 0')
clean_colnames(df)
df.head()

,authors,date,journal-name,journal-name-abbrv.,pmid,publication-type,title
0,Elkiran T|Harputluoglu H|Yasar U|Babaoglu MO|D...,2007 Jan-Feb,Methods and findings in experimental and clini...,Methods Find Exp Clin Pharmacol,17344941,Clinical Trial|Journal Article,Differential alteration of drug-metabolizing e...
1,Severins N|Mensink RP|Plat J,2015 Feb,"Nutrition, metabolism, and cardiovascular dise...",Nutr Metab Cardiovasc Dis,25456153,Journal Article|Randomized Controlled Trial,Effects of lutein-enriched egg yolk in butterm...
2,Li C|Zuo C|Deng G|Kuang R|Yang Q|Hu C|Sheng O|...,2013,PloS one,PLoS One,23922960,Journal Article,Contamination of bananas with beauvericin and ...
3,Güneser S|Atici A|Cengizler I|Alparslan N,1996 May-Jun,Allergologia et immunopathologia,Allergol Immunopathol (Madr),8766742,Journal Article,Inhalant allergens: as a cause of respiratory ...
4,Brvar M|Bunc M,2009 Sep 9,Cases journal,Cases J,19918387,Journal Article,High-degree atrioventricular block in acute et...


In [10]:
df = df.where((pd.notnull(df)), None)

df.rename(columns={
    'authors': 'authors',
    'journal-name': 'journal_name',
    'journal-name-abbrv.': 'journal_name_abbr',
    'publication-type':'publication_type',
},inplace=True)

data_list= list(df.T.to_dict().values())
data_list[0]

{'authors': 'Elkiran T|Harputluoglu H|Yasar U|Babaoglu MO|Dincel AK|Altundag K|Ozisik Y|Guler N|Bozkurt A',
 'date': '2007 Jan-Feb',
 'journal_name': 'Methods and findings in experimental and clinical pharmacology',
 'journal_name_abbr': 'Methods Find Exp Clin Pharmacol',
 'pmid': 17344941,
 'publication_type': 'Clinical Trial|Journal Article',
 'title': 'Differential alteration of drug-metabolizing enzyme activities after cyclophosphamide/adriamycin administration in breast cancer patients.'}

### Chemicals

In [11]:
fchem = pd.read_csv(DATA/'food-chemical.tsv', sep='\t')
pubchems = set(fchem['pubchem-id'])

## B. Associations

In [12]:
food_dis = pd.read_csv(DATA/'food-disease.tsv', sep='\t', encoding='utf-8')
food_dis.drop_duplicates(['food-id', 'disease-id', 'association', 'pmid'], inplace=True)
food_dis = food_dis.where((pd.notnull(food_dis)), None)
food_dis['association'] = food_dis['association'].apply(str.lower)
food_dis = food_dis.loc[:, ['food-id', 'disease-id', 'association', 'pmid']]
food_dis['pmid'] = food_dis['pmid'].apply(str)
food2dis = defaultdict(lambda: set(),
                       food_dis.groupby('food-id').agg(lambda s: set(s))['disease-id'].to_dict())
food_dis.head(1)

,food-id,disease-id,association,pmid
0,Plant ID:4045,MESH:D004487,negative,7398283


In [13]:
chem_dis = pd.read_csv(DATA/'chemical-disease.tsv', sep='\t', encoding='utf-8', index_col='Unnamed: 0')
chem_dis = chem_dis.groupby(['pubchem-id', 'disease-id']).agg(lambda s: sorted(s)[-1]).reset_index()
dis2chem = defaultdict(lambda: set(),
                       chem_dis.groupby('disease-id').agg(lambda s: set(s))['pubchem-id'].to_dict())
chem2dis = defaultdict(lambda: set(),
                       chem_dis.groupby('pubchem-id').agg(lambda s: set(s))['disease-id'].to_dict())

chem_dis.head(1)

,pubchem-id,disease-id,type
0,19,MESH:D007674,therapeutic


In [14]:
food_chem = pd.read_csv(DATA/'food-chemical.tsv', sep='\t', encoding='utf-8')
food2chem = defaultdict(lambda: set(),
                        food_chem.groupby('food-id').agg(lambda s: set(s))['pubchem-id'].to_dict())

food_chem.head(1)

,food-id,pubchem-id,content,reference
0,Plant ID:100170,5280378,Detected but not quantified,KNApSAcK


In [55]:
gene_dis = pd.read_csv(DATA/'gene-disease.tsv', sep='\t', encoding='utf-8', index_col='Unnamed: 0')
dis2gene = defaultdict(lambda: set(),
                        gene_dis.groupby('disease-id').agg(lambda s: set(s))['gene-id'].to_dict())
gene2dis = defaultdict(lambda: set(),
                        gene_dis.groupby('gene-id').agg(lambda s: set(s))['disease-id'].to_dict())

gene_dis.head()

,disease-id,gene-id,source
0,MESH:C000591739,1588,ORPHANET
1,MESH:C000596385,26504,UNIPROT
2,MESH:C000598645,9499,CTD_human;ORPHANET;UNIPROT
3,MESH:C000600608,29940,ORPHANET
4,MESH:C000600608,113189,CTD_human;ORPHANET;UNIPROT


In [40]:
chem_gene = pd.read_csv(DATA/'chemical-gene.tsv', sep='\t', encoding='utf-8')
chem2gene = defaultdict(lambda: set(),
                        chem_gene.groupby('pubchem-id').agg(lambda s: set(s))['gene-id'].to_dict())
gene2chem = defaultdict(lambda: set(),
                        chem_gene.groupby('gene-id').agg(lambda s: set(s))['pubchem-id'].to_dict())
chem_gene.head()

,pubchem-id,gene-id,interaction-actions,source
0,19,351,affects^binding|decreases^reaction,CTD
1,19,4313,decreases^activity|decreases^reaction|decrease...,CTD
2,51,3091,affects^binding|affects^cotreatment|increases^...,CTD
3,51,7428,affects^binding|affects^cotreatment|increases^...,CTD
4,51,8290,affects^cotreatment|decreases^methylation|incr...,CTD


### 1. Food-Page

##### Food-disease

In [18]:
# Curated
neg_fd = food_dis[food_dis.association == 'negative'].\
    groupby(['food-id', 'disease-id']).agg(lambda s: '|'.join(s))[['pmid']].reset_index(())
neg_fd.reset_index(inplace=True)
neg_fd.rename(columns={'pmid': 'negative-pmid'}, inplace=True)

pos_fd = food_dis[food_dis.association == 'positive'].\
    groupby(['food-id', 'disease-id']).agg(lambda s: '|'.join(s))[['pmid']]
pos_fd.reset_index(inplace=True)
pos_fd.rename(columns={'pmid': 'positive-pmid'}, inplace=True)

food_dis_curated = pos_fd.merge(neg_fd, on=['food-id', 'disease-id'], how='outer').fillna('')

# Inferred
fd_inferred = list()
for food, chemicals in food2chem.items():
    for chem in chemicals:
        for dis in chem2dis[chem]:
            fd_inferred.append([food, dis, str(chem)])
            
fd_inferred = pd.DataFrame(fd_inferred, 
                           columns=['food-id', 'disease-id', 'pubchem-id'])
fd_inferred = fd_inferred.groupby(['food-id', 'disease-id']).agg(lambda s: '|'.join(s)).reset_index()
del pos_fd, neg_fd

fd_inferred.head(2)

,food-id,disease-id,pubchem-id
0,Plant ID:100170,MESH:D002583,5280378
1,Plant ID:100170,MESH:D058186,5280378


In [20]:
food_dis_db = food_dis_curated.merge(fd_inferred, on=['food-id', 'disease-id'], how='outer').fillna('')
food_dis_db['weight'] = food_dis_db.apply(lambda row: len(row['positive-pmid'].split('|')) +
                                          len(row['positive-pmid'].split('|')), 1)
del fd_inferred, food_dis_curated

food_dis_db.head()

,food-id,disease-id,positive-pmid,negative-pmid,pubchem-id,weight
0,AlcoholicBev ID:17,MESH:D001523,7195588,,,2
1,AlcoholicBev ID:17,MESH:D003866,6798614,,,2
2,AlcoholicBev ID:18,MESH:D005756,11848298,,,2
3,AlcoholicBev ID:18,MESH:D013272,17295722,9615879|17295722,,2
4,AlcoholicBev ID:2,MESH:C537766,25604939,,,2


##### Food-chemical

In [21]:
food_chem_db = food_chem.where((pd.notnull(food_chem)), None)
food_chem_db.head()

,food-id,pubchem-id,content,reference
0,Plant ID:100170,5280378,Detected but not quantified,KNApSAcK
1,Plant ID:100170,44260113,Detected but not quantified,KNApSAcK
2,Plant ID:100506,1150,Detected but not quantified,KNApSAcK
3,Plant ID:100506,5202,Detected but not quantified,KNApSAcK
4,Plant ID:100506,5372945,Detected but not quantified,KNApSAcK


#### Food-gene

In [22]:
# Inferred throuch chemicals
fg_inf_chem = list()
for food, chemlst in food2chem.items():
    for chem in chemlst:
        for gene in chem2gene[chem]:
            fg_inf_chem.append([food, str(chem), gene])
            
fg_inf_chem = pd.DataFrame(fg_inf_chem, columns=['food-id', 'pubchem-id', 'gene-id'])
fg_inf_chem = fg_inf_chem.groupby(['food-id', 'gene-id']).agg(lambda s: '|'.join(s)).reset_index()
fg_inf_chem.head(2)

,food-id,gene-id,pubchem-id
0,Plant ID:100170,196,5280378
1,Plant ID:100170,581,5280378


In [29]:
# Inferred throuch diseases
fg_inf_dis = list()
for food, dislst in food2dis.items():
    for dis in dislst:
        for gene in dis2gene[dis]:
            fg_inf_dis.append([food, dis, gene])
            
fg_inf_dis = pd.DataFrame(fg_inf_dis, columns=['food-id', 'disease-id', 'gene-id'])
fg_inf_dis = fg_inf_dis.groupby(['food-id', 'gene-id']).agg(lambda s: '|'.join(s)).reset_index()
fg_inf_dis.head(2)

,food-id,gene-id,disease-id
0,AlcoholicBev ID:17,2,MESH:D003866
1,AlcoholicBev ID:17,15,MESH:D003866


In [30]:
food_gene_db = fg_inf_dis.merge(fg_inf_chem, on=['food-id', 'gene-id'], how='outer').fillna('')
food_gene_db.rename(columns={'disease-id': 'via-diseases', 'pubchem-id': 'via-chemicals'}, inplace=True)

del fg_inf_chem, fg_inf_dis

In [37]:
# TAKES LOT OF TIME!
# food_gene_db['weight'] = food_gene_db.apply(lambda row: len(row['via-diseases'].split('|')) +
#                                                         len(row['via-chemicals'].split('|')), 1)

KeyboardInterrupt: 

### 2. Disease page

#### Disease-gene

In [38]:
# Curated
dg_curated = gene_dis.copy(deep=True)

# Inferred through chemicals
dg_inferred = list()
for disease, chemlst in dis2chem.items():
    for chem in chemlst:
        for gene in chem2gene[chem]:
            dg_inferred.append([disease, str(chem), gene])
            
dg_inferred = pd.DataFrame(dg_inferred, columns=['disease-id', 'pubchem-id', 'gene-id'])
dg_inferred = dg_inferred.groupby(['disease-id', 'gene-id']).agg(lambda s: '|'.join(s)).reset_index()
dg_inferred.head(2)

,disease-id,gene-id,pubchem-id
0,MESH:C531617,14,14985
1,MESH:C531617,35,14985


In [39]:
# Merge
disease_gene_db = dg_curated.merge(dg_inferred, on=['disease-id', 'gene-id'], how='outer').fillna('')
disease_gene_db.rename(columns={'pubchem-id': 'via-chemicals'}, inplace=True)
del dg_inferred, dg_curated

disease_gene_db.head(1)

,disease-id,gene-id,source,via-chemicals
0,MESH:C000591739,1588,ORPHANET,


#### Disease-chemical

In [51]:
# Curated
chem_dis_curated = chem_dis.copy(deep=True)
chem_dis_curated['pubchem-id'] = chem_dis_curated['pubchem-id'].apply(str)

# Inferred through genes
dc_inferred = list()
for disease, genelst in dis2gene.items():
    for gene in genelst:
        for chem in gene2chem[gene]:
            dc_inferred.append([disease, str(chem), str(gene)])
            
dc_inferred = pd.DataFrame(dc_inferred, columns=['disease-id', 'pubchem-id', 'gene-id'])
dc_inferred = dc_inferred.groupby(['disease-id', 'pubchem-id']).agg(lambda s: '|'.join(s)).reset_index()
dc_inferred.head(2)

,disease-id,pubchem-id,gene-id
0,MESH:C000591739,10680,1588
1,MESH:C000591739,14403,1588


In [52]:
# Merge
disease_chem_db = dc_inferred.merge(chem_dis_curated, on=['disease-id', 'pubchem-id'], how='outer').fillna('')
disease_chem_db.rename(columns={'gene-id': 'via-genes'}, inplace=True)
del disease_chem_db['type']
del dc_inferred, chem_dis_curated

disease_chem_db.head(1)

,disease-id,pubchem-id,via-genes
0,MESH:C000591739,10680,1588


### 3. Gene page

#### Gene-chemical

In [75]:
# Curated
chem_gene_curated = chem_gene.copy(deep=True)
chem_gene_curated['gene-id'] = chem_gene_curated['gene-id'].apply(str)
chem_gene_curated['pubchem-id'] = chem_gene_curated['pubchem-id'].apply(str)

# Inferred through diseases
gc_inferred = list()
for gene, dislst in gene2dis.items():
    for dis in dislst:
        for chem in dis2chem[dis]:
            gc_inferred.append([str(gene), str(chem), dis])
            
gc_inferred = pd.DataFrame(gc_inferred, columns=['gene-id', 'pubchem-id', 'disease-id'])
gc_inferred = gc_inferred.groupby(['gene-id', 'pubchem-id']).agg(lambda s: '|'.join(s)).reset_index()
gc_inferred.head(2)

,gene-id,pubchem-id,disease-id
0,10,10208,MESH:D056486
1,10,10212,MESH:D056486


In [74]:
# Merge
chem_gene_db = gc_inferred.merge(chem_gene_curated, on=['gene-id', 'pubchem-id'], how='outer').fillna('')
chem_gene_db.rename(columns={'disease-id': 'via-diseases'}, inplace=True)
del chem_gene_db['interaction-actions'], chem_gene_db['source']
del gc_inferred, chem_gene_curated

chem_gene_db.head(1)

,gene-id,pubchem-id,via-diseases
0,10,10208,MESH:D056486
